# Types

This page discusses different approaches to annotating code structures.

For more details check the [Specification for the Python type system](https://typing.python.org/en/latest/spec/index.html).

**Note:** This notebook uses a [Command Kernel](https://github.com/fedorkobak/command_kernel) with implemented `# mypy` and `# pyright` commands. These commands apply the corresponding linter to the file created from the cell content. By default, the python interpreter is applied to the cell content. 

## Union

If a value can take multiple types, you have to annotate it as an enumeration of types with `|` symbol. Using the syntax `typing.Union[<type 1>, <type 2>, ...] will have a similar result.

---

The following cell shows that to variables annotated as `int | str` you can assign both types.

In [27]:
# mypy
val: int | str = 10
val2: int | str = "hello"

Success: no issues found in 1 source file(B


However, the attempt to assign a float fails.

In [28]:
# mypy
val3: int | str = 3.3

/tmp/tmpktyodpvp:1: error:(B Incompatible types in assignment (expression has type (B"float"(B, variable has type (B"int | str"(B)  (B[assignment](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

**Note:** The `typing.Union` syntax can look a bit inconvenient if you compare it to enumeration by `|`, but it has advantage of possibility to list awailable types in the different place.

For example, the following cell shows how you can define `typing.Union` with types defined as the elements of the list.

In [25]:
# mypy
import typing
types = [int, str, float]
typing.Union[*types]

Success: no issues found in 1 source file(B


## Void functions

If function doesn't return anything you have to specify `None` as type of output. In other cases, type analisis tools will allow you to assign the function's returns to any value - which is incorrect behavior.

---

The following cell defines the void function and assigns its result to the variable - which is nonsence.

In [1]:
# mypy
def fun():
    print("test")

val = fun()

Success: no issues found in 1 source file(B


But `mypy` sees no problem here - just because the output of the function is not defined.

In contrast, the following cell creates the same file, but function return is annotated as `None`.

In [2]:
# mypy
def fun(val: int) -> None:
    print("test")

val = fun(3)

/tmp/tmpxn3tajvv:4: error:(B (B"fun"(B does not return a value (it only ever returns None)  (B[func-returns-value](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

As result `mypy` returns corresponding error.

## No return

In case some can be stopped during execution without returning anything outside - you must use `typing.NoReturn` as return type for the function. `None` is not suitable here because it means that the variable to which the return value is assigned must accept the type `None` - but it's not correct if the exception or some other termination function doesn't return anything.

---

The following cell creates a function that can return `int` in some cases or just raise the exception.

In [3]:
# mypy
def fun(val: int) -> int | None:
    if val < 0:
        raise Exception("test")
    return 5

val: int = fun(5)

/tmp/tmp07z0no5g:6: error:(B Incompatible types in assignment (expression has type (B"int | None"(B, variable has type (B"int"(B)  (B[assignment](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

As a result, trying to assign the result of the `fun` to an integer value will result in an error.

But if you use `typing.NoReturn` everything works fine.

In [4]:
# mypy
from typing import NoReturn

def fun(val: int) -> int | NoReturn:
    if val < 0:
        raise Exception("test")
    return 5

val: int = fun(5)

Success: no issues found in 1 source file(B


## Tuple

There are significant differences between annotations for lists or sets and annotations for tuples. In tuples, you have to define the type of each element individually, which means you need to count the number of elements in the tuple. However, for lists or sets, it's sufficient to annotate the types that can be stored in the collection.

---

The following cell shows that the annotation `tuple[int, bool, float, str]` does not correspond to the value `(10, True, 3.0)`.

In [6]:
# mypy
val: tuple[int, bool, float, str] = (10, True, 3.0)

/tmp/tmpk4li_hxr:1: error:(B Incompatible types in assignment (expression has type (B"tuple[int, bool, float]"(B, variable has type (B"tuple[int, bool, float, str]"(B)  (B[assignment](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

It expects another `str` value as the last element of the `tuple`. The following cell compares the tuple `(10, True, 3.0, "hello")` with the given annotation.

In [8]:
# mypy
val: tuple[int, bool, float, str] =  (10, True, 3.0, "hello")

Success: no issues found in 1 source file(B


Now everything is fine.

## Any type

Sometimes, you'll encounter cases in which an object can take any type. In most cases, you can just ignore the type. However, there are reasons why you should have the option to declare an expression can have any type:

- To show that any type is a deliberate decision.
- To have an option for cases where a type must be specified, such as the type of keys in a dictionary or the type of a particular element in a tuple.

---

Consider a function that requires a dict with `float` keys, but doesn't care about the actual type of the dictionary's values:

In [ ]:
# mypy
from typing import Any
def max_key(inp_dict: dict[float]):
    def selector(key: float): return inp_dict.get(key)
    return max(inp_dict, key=selector)

max_key({10: 3, 7: "hello"})

/tmp/tmpc2htl_zf:2: error:(B (B"dict"(B expects 2 type arguments, but 1 given  (B[type-arg](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

`mypy` produces ouput that notes that the `dict` type annotation requires two arguments. Therefore, it is supposed to be annotated as `Any`.

In [13]:
# mypy
from typing import Any
def max_key(inp_dict: dict[float, Any]) -> Any:
    def selector(key: float) -> Any: return inp_dict.get(key)
    return max(inp_dict, key=selector)

max_key({10: 3, 7: "hello"})

Success: no issues found in 1 source file(B


## Sequence

With `typing.Sequence`, you can annotate any subscriptable type and those that have a defined `__len__` dunder.

---

The following cell show the comparison of the `list` and the `tuple` with the `Sequence` annotation.

In [19]:
# mypy
import typing
val1: typing.Sequence[str | bool] = (True, "hello", False)
val2: typing.Sequence[str | bool] = [True, "hello", False]

Success: no issues found in 1 source file(B


Everything works fine. But the following cell shows that the `set` doesn't refer to the `Sequence`.

In [21]:
# mypy
import typing
val: typing.Sequence[str | bool] = {True, "hello", False},

/tmp/tmpm7pdikhl:2: error:(B Incompatible types in assignment (expression has type (B"tuple[set[object]]"(B, variable has type (B"Sequence[str | bool]"(B)  (B[assignment](B
Found 1 error in 1 file (checked 1 source file)(B


: 1

## Typed dictionaries

By inheriting the `typing.TypeDict`, you can define a type that behaves like a typed dictionary. For each potential key, you can specify the expected value type.

There are following important details associated with typed dict:

- By default, you can add any keys to a `TypedDict` instance. It has to be makred as `closed` to prevent this behavior. 
- By default, all attributes specified in the definition of the `TypeDict` haires must be provided, during the creation of an instance. You can regulate this behaviour using:
    - The `total` argument in call definition. 
    - The `typing.Required[]` or `typing.NotRequired[]` qualifiers for the attributes.
- The `extra_items` argument allows you to specify the type of extra values that are not specified in the defition of the `TypedDict`.
- You can define a generic `TypedDict`. This means that you can specify the type of some elements when creating the instance.

Check more in the [`TypedDict`](https://typing.python.org/en/latest/spec/typeddict.html) page.

---

Consider the following exmaple: The cell defines a class whose instances will behave exactly like a dictionary. However, the value udner the "a" key have to be an integer, and the value under the "b" key have to be a string.

In [22]:
# mypy
from typing import TypedDict

class MyDict(TypedDict):
    a: int
    b: str

MyDict(a="hello", b=20)

/tmp/tmpocl3wgaj:7: error:(B Incompatible types (expression has type (B"str"(B, TypedDict item (B"a"(B has type (B"int"(B)  (B[typeddict-item](B
/tmp/tmpocl3wgaj:7: error:(B Incompatible types (expression has type (B"int"(B, TypedDict item (B"b"(B has type (B"str"(B)  (B[typeddict-item](B
Found 2 errors in 1 file (checked 1 source file)(B


: 1

## Generics

A generic is a type that can be parametrized with other types. The simpliest and probably the most common generic type is `list[int]`, which means that you are dealing with the list of exactly integers.

In this context, the list annotation is parametrized with `int`, meaning that any linter or completor treat the elements of the list as integer values.

For more details check the: 

- [Generics](https://typing.python.org/en/latest/reference/generics.html) page of the official documentation.
- [Generics](https://docs.python.org/3/library/typing.html#generics) section of the `typing` package.
- [Generics](https://typing.python.org/en/latest/spec/generics.htm) section in the specification of the python typing system.
- [Generics](typing/generics.ipynb) page on this site.

Most cases of annotations using generics have their own subsection on the page. This section explains the concept of generics and how to create the custom ones.

## Annotated

The `typing.Annotated` allows the specification of metadata for a variable. This metadata is typically used by frameworks to build a specific patterns that allow to specify how frame work have to deal with the variable.

The following tools are usfull when working with Annotated:

- Define the metadate with `Annotated[<type>, <metadata1>, <metadata2>, ...]`.
- To load the annotations for the object with metadata use `typing.get_type_hints(<object>, include_extras=True)`.
- To access the metadata use, the `__metadata__` attribute of the `typing.Annotated` object.

---

The following code defines the `Example` dataclass with `x` attibute to have `"positive"` as metadata.

In [17]:
from dataclasses import dataclass
from typing import Annotated, get_type_hints

@dataclass
class Example():
    x: Annotated[int, "positive"]

The following cell shows the output of the `typing.get_type_hints` for the `Example` class.

In [25]:
hints = get_type_hints(Example, include_extras=True)
hints

{'x': typing.Annotated[int, 'positive']}

And the way to access exactly metadata.

In [27]:
hints['x'].__metadata__

('positive',)

Consider how metadata can be used. The `process` function checks whether the `x` attribute of the passed object has been annotated as positive.

In [28]:
def process(inp):
    hints = get_type_hints(type(inp), include_extras=True)
    if hints['x'].__metadata__[0] == "positive" and inp.x < 0:
        print("Warning")

Therefore, the warning will be printed for the `Example` instance initialised with a negative `x`.

In [29]:
process(Example(x=-2))

Warning


But, if the class where `x` annotated with any other value, everything will be fine.

In [23]:
@dataclass
class Example2():
    x: Annotated[int, "any"]

process(Example2(x=-2))